# Quantum Process Tomography

* **Last Updated:** Jan 24, 2019
* **Requires:**
  * qiskit-terra 0.7
  * qiskit-aer 0.1

In [1]:
# Needed for functions
import numpy as np
import time

# Import QISKit classes
import qiskit
from qiskit import QuantumRegister, QuantumCircuit, Aer
from qiskit.quantum_info import state_fidelity
from qiskit.tools.qi.qi import outer

# Tomography functions
import sys, os
sys.path.append(os.path.abspath(os.path.join('../qiskit_ignis')))
import tomography as tomo

## 1-qubit process tomography example

In [2]:
# Process tomography of a Hadamard gate
q = QuantumRegister(1)
circ = QuantumCircuit(q)
circ.h(q[0])

# Run circuit on unitary simulator to find ideal unitary
job = qiskit.execute(circ, Aer.get_backend('unitary_simulator'))
ideal_unitary = job.result().get_unitary(circ)
# convert to Choi-matrix in column-major convention
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
qpt_circs = tomo.process_tomography_circuits(circ, q)
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=4000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_counts = tomo.tomography_data(job.result(), qpt_circs)
qpt_counts

{(('Zp',), ('X',)): {'0': 4000},
 (('Zp',), ('Y',)): {'1': 2043, '0': 1957},
 (('Zp',), ('Z',)): {'1': 1955, '0': 2045},
 (('Zm',), ('X',)): {'1': 4000},
 (('Zm',), ('Y',)): {'1': 1989, '0': 2011},
 (('Zm',), ('Z',)): {'1': 2024, '0': 1976},
 (('Xp',), ('X',)): {'1': 1982, '0': 2018},
 (('Xp',), ('Y',)): {'1': 2014, '0': 1986},
 (('Xp',), ('Z',)): {'0': 4000},
 (('Yp',), ('X',)): {'1': 1997, '0': 2003},
 (('Yp',), ('Y',)): {'1': 4000},
 (('Yp',), ('Z',)): {'1': 1958, '0': 2042}}

In [3]:
# Extract fitter data from tomography counts
data, basis = tomo.fitter_data(qpt_counts)

# MLE Least-Squares tomographic reconstruction
t = time.time()
choi_mle = tomo.process_mle_fit(data, basis)
print('MLE Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_mle / 2))

# CVXOPT Semidefinite-Program tomographic reconstruction
t = time.time()
choi_cvx = tomo.fitters.cvx_fit(data, basis, PSD=True, trace_preserving=True, trace=2)
choi_cvx = tomo.process_cvx_fit(data, basis)
print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_cvx / 2))

MLE Fitter
fit time: 0.16478204727172852
fit fidelity: 0.9927145287905677

CVXOPT Fitter
fit time: 0.07139205932617188
fit fidelity: 0.9999595309349295


## 1-qubit process tomography of two-qubit swap gate

We will prepare qubit-0 and measure qubit-1 so the reconstructed channel should be an identity

In [4]:
# Process tomography of a Hadamard gate
q = QuantumRegister(2)
circ = QuantumCircuit(q)
circ.swap(q[0], q[1])

# Ideal channel is a unitary
ideal_unitary = np.eye(2)
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
# We use the optional prepared_qubits kwarg to specify that the prepared qubit was different to measured qubit
qpt_circs = tomo.process_tomography_circuits(circ, q[1], prepared_qubits=q[0])
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=2000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_counts = tomo.tomography_data(job.result(), qpt_circs)
qpt_counts

{(('Zp',), ('X',)): {'1': 987, '0': 1013},
 (('Zp',), ('Y',)): {'1': 985, '0': 1015},
 (('Zp',), ('Z',)): {'0': 2000},
 (('Zm',), ('X',)): {'1': 975, '0': 1025},
 (('Zm',), ('Y',)): {'1': 978, '0': 1022},
 (('Zm',), ('Z',)): {'1': 2000},
 (('Xp',), ('X',)): {'0': 2000},
 (('Xp',), ('Y',)): {'1': 1027, '0': 973},
 (('Xp',), ('Z',)): {'1': 969, '0': 1031},
 (('Yp',), ('X',)): {'1': 1020, '0': 980},
 (('Yp',), ('Y',)): {'0': 2000},
 (('Yp',), ('Z',)): {'1': 1010, '0': 990}}

In [5]:
# Extract fitter data from tomography counts
data, basis = tomo.fitter_data(qpt_counts)

# MLE Least-Squares tomographic reconstruction
t = time.time()
choi_mle = tomo.process_mle_fit(data, basis)
print('MLE Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_mle / 2))

# CVXOPT Semidefinite-Program tomographic reconstruction
t = time.time()
choi_cvx = tomo.fitters.cvx_fit(data, basis, PSD=True, trace_preserving=True, trace=2)
choi_cvx = tomo.process_cvx_fit(data, basis)
print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_cvx / 2))

MLE Fitter
fit time: 0.0012328624725341797
fit fidelity: 0.9820372305628304

CVXOPT Fitter
fit time: 0.0461270809173584
fit fidelity: 0.9999537962913683


## 2-Qubit entangling circuit

In [6]:
# Bell-state entangling circuit
q = QuantumRegister(2)
circ = QuantumCircuit(q)
circ.h(q[0])
circ.cx(q[0], q[1])

# Run circuit on unitary simulator to find ideal unitary
job = qiskit.execute(circ, Aer.get_backend('unitary_simulator'))
ideal_unitary = job.result().get_unitary(circ)
# convert to Choi-matrix in column-major convention
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
qpt_circs = tomo.process_tomography_circuits(circ, q)
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=2000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_counts = tomo.tomography_data(job.result(), qpt_circs)

# Extract fitter data from tomography counts
data, basis = tomo.fitter_data(qpt_counts)


t = time.time()
choi_mle = tomo.process_mle_fit(data, basis)
print('MLE Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 4, choi_mle / 4))

t = time.time()
choi_cvx = tomo.process_cvx_fit(data, basis)
print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 4, choi_cvx / 4))

MLE Fitter
fit time: 0.028927087783813477
fit fidelity: 0.9840922722224257

CVXOPT Fitter
fit time: 1.2832000255584717
fit fidelity: 0.999908669144743


## Using SIC-POVM preparation basis

In [7]:
# Process tomography of a Hadamard gate
q = QuantumRegister(1)
circ = QuantumCircuit(q)
circ.h(q[0])

# Run circuit on unitary simulator to find ideal unitary
job = qiskit.execute(circ, Aer.get_backend('unitary_simulator'))
ideal_unitary = job.result().get_unitary(circ)
# convert to Choi-matrix in column-major convention
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
qpt_circs = tomo.process_tomography_circuits(circ, q, prep_labels='SIC', prep_basis='SIC')
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=2000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_counts = tomo.tomography_data(job.result(), qpt_circs)
qpt_counts

{(('S0',), ('X',)): {'0': 2000},
 (('S0',), ('Y',)): {'1': 966, '0': 1034},
 (('S0',), ('Z',)): {'1': 1036, '0': 964},
 (('S1',), ('X',)): {'1': 1332, '0': 668},
 (('S1',), ('Y',)): {'1': 1026, '0': 974},
 (('S1',), ('Z',)): {'1': 60, '0': 1940},
 (('S2',), ('X',)): {'1': 1342, '0': 658},
 (('S2',), ('Y',)): {'1': 166, '0': 1834},
 (('S2',), ('Z',)): {'1': 1503, '0': 497},
 (('S3',), ('X',)): {'1': 1351, '0': 649},
 (('S3',), ('Y',)): {'1': 1823, '0': 177},
 (('S3',), ('Z',)): {'1': 1446, '0': 554}}

In [8]:
# Extract fitter data from tomography counts
data, basis = tomo.fitter_data(qpt_counts, prep_basis='SIC')

# MLE Least-Squares tomographic reconstruction
t = time.time()
choi_mle = tomo.process_mle_fit(data, basis)
print('MLE Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_mle / 2))

# CVXOPT Semidefinite-Program tomographic reconstruction
t = time.time()
choi_cvx = tomo.process_cvx_fit(data, basis)

print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_cvx / 2))

MLE Fitter
fit time: 0.0009930133819580078
fit fidelity: 0.9937348052238946

CVXOPT Fitter
fit time: 0.025810956954956055
fit fidelity: 0.99903075629802
